# Technical offer, Lot 1 TO 2
----

CEA Consortium, in response to Self-describing NetCDF format for IMAS data structures - TO 2 for LOT 1
Technical Specifications reference: 5RYPMA

This Jupyter notebook is an	annexe of EPSYL Technical offer.

In [1]:
from netCDF4 import Dataset
import numpy as np

## Exploration of the saving methods

This sections test three identified saving methods to the NetCDF data format.

### Generation of dummy data

We generate two large matrix of differents size to test NetCDF capacity to save heterogeneous data.

In [2]:
# Grids to use with the ncdump command as their are smaller and easy to read
GRID_1 = np.arange(0,5, dtype=float)
GRID_2 = np.arange(0,10, dtype=float)

In [3]:
# Grids to use to test and explore the NetCDF format
GRID_1 = np.arange(0,100_000, dtype=float)
GRID_2 = np.arange(0,200_000, dtype=float)

In [4]:
isinstance( GRID_1[0], np.float64 )

True

### Tensorization

The tensorization method creates a unique variable psi and two dimensions were used: itime and psi_dim. The itime dimensions represent the number of time_slice measured and psi_dim is dimension is the number of measurements, represented as N on the following figures. 

In [5]:
%%timeit
# Create the NetCDF object
rootgrp = Dataset("tensorisation.nc", "w", format="NETCDF4", persist=True)

# Define the dimensions
itime = rootgrp.createDimension("itime", 2) 
size1 = rootgrp.createDimension("psi_dim", 0) # 0 mean unlimited size, this is needed as the two grids have multiple size

# Create the variable
grid = rootgrp.createVariable("psi", "f4", ( "itime", "psi_dim")) 

# Set data
grid[0] = GRID_1 # Padding automaticaly added 
grid[1] = GRID_2

rootgrp.close()

2.03 s ± 90.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
!du -h tensorisation.nc

11M	tensorisation.nc


In [7]:
%%timeit
# Create the NetCDF object
rootgrp = Dataset("tensorisation_compression.nc", "w", format="NETCDF4", persist=True, compression='zlib', complevel=9)

# Define the dimensions
itime = rootgrp.createDimension("itime", 2) 
size1 = rootgrp.createDimension("psi_dim", 0) # 0 mean unlimited size, this is needed as the two grids have multiple size

# Create the variable
grid = rootgrp.createVariable("psi", "f4", ( "itime", "psi_dim")) 

# Set data
grid[0] = GRID_1 # Padding automaticaly added 
grid[1] = GRID_2

rootgrp.close()

2.15 s ± 79.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
!du -h tensorisation_compression.nc

11M	tensorisation_compression.nc


### Variabilization

In this method, variables are created instead of dimension to store data. This method is called “variabilization” as it multiplies the number of variables.  Each of those variables had their own dimensions to avoid padding.

In [11]:
%%timeit
rootgrp = Dataset("variabilisation.nc", "w", format="NETCDF4", persist=True)

# Define the dimensions
itime = rootgrp.createDimension("itime", 2) 
size1 = rootgrp.createDimension("psi_1_dim", len(GRID_1)) 
size2 = rootgrp.createDimension("psi_2_dim", len(GRID_2))

# Create variables
grid1 = rootgrp.createVariable("time_slice_1__profiles_1d__psi", "f4", ( "itime", "psi_1_dim"))  
grid2 = rootgrp.createVariable("time_slice_2__profiles_1d__psi", "f4", ( "itime", "psi_2_dim"))  

# Set data
grid1[:] = GRID_1 # No padding
grid2[:] = GRID_2

rootgrp.close()

18.8 ms ± 4.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
!du -h variabilisation.nc

2,3M	variabilisation.nc


In [13]:
%%timeit
rootgrp = Dataset("variabilisation_compression.nc", "w", format="NETCDF4", persist=True, compression='zlib', complevel=9)

# Define the dimensions
itime = rootgrp.createDimension("itime", 2) 
size1 = rootgrp.createDimension("psi_1_dim", len(GRID_1)) 
size2 = rootgrp.createDimension("psi_2_dim", len(GRID_2))

# Create variables
grid1 = rootgrp.createVariable("time_slice_1__profiles_1d__psi", "f4", ( "itime", "psi_1_dim"))  
grid2 = rootgrp.createVariable("time_slice_2__profiles_1d__psi", "f4", ( "itime", "psi_2_dim"))  

# Set data
grid1[:] = GRID_1 # No padding
grid2[:] = GRID_2

rootgrp.close()

16.9 ms ± 3.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
!du -h variabilisation_compression.nc

2,3M	variabilisation_compression.nc


### Compression 
The compression method takes a multidimensional array and flatten it in a simple flat vector. To be able to reconstruct the original information, a companion array is created, containing the index of each original array flattened in the flat vector.

In [15]:
%%timeit
rootgrp = Dataset("compression.nc", "w", format="NETCDF4", persist=True)

variable = rootgrp.createDimension("variable", len(GRID_1) + len(GRID_2))
index_var = rootgrp.createDimension("index_var", 2)

grid_val = rootgrp.createVariable("psi_value", "f4", ("variable")) 
index_var = rootgrp.createVariable("psi_index", "f4", ("index_var")) 

grid_val[:] = np.hstack([GRID_1, GRID_2])
index_var[:] = np.array([len(GRID_1), len(GRID_1)+len(GRID_2)])

rootgrp.close()

8.3 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
!du -h compression.nc

1,2M	compression.nc


In [17]:
%%timeit
rootgrp = Dataset("compression_compression.nc", "w", format="NETCDF4", persist=True, compression='zlib', complevel=9)

variable = rootgrp.createDimension("variable", len(GRID_1) + len(GRID_2))
index_var = rootgrp.createDimension("index_var", 2)

grid_val = rootgrp.createVariable("psi_value", "f4", ("variable")) 
index_var = rootgrp.createVariable("psi_index", "f4", ("index_var")) 

grid_val[:] = np.hstack([GRID_1, GRID_2])
index_var[:] = np.array([len(GRID_1), len(GRID_1)+len(GRID_2)])

rootgrp.close()

8.14 ms ± 2.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
!du -h compression_compression.nc

1,2M	compression_compression.nc


### Saving methods conclusion

Here are the file size for each saving methods

In [19]:
print(f"For grid size 1 : {len(GRID_1)} and grid size 2 : {len(GRID_2)}")
!du -h tensorisation.nc variabilisation.nc compression.nc tensorisation_compression.nc variabilisation_compression.nc compression_compression.nc

For grid size 1 : 100000 and grid size 2 : 200000
11M	tensorisation.nc
2,3M	variabilisation.nc
1,2M	compression.nc
11M	tensorisation_compression.nc
2,3M	variabilisation_compression.nc
1,2M	compression_compression.nc




---

## NetCDF and Xarray

We test NetCDF read / modify /write operation for all IDSs listed in the DD using just the python xarray library

In [20]:
import xarray as xr

### Read a NetCDF file with Xarray

In this subsection, NetCDF files are read using the Xarray libraries

In [21]:
ds_disk = xr.load_dataset("tensorisation.nc")
ds_disk 

<xarray.Dataset>
Dimensions:  (itime: 2, psi_dim: 200000)
Dimensions without coordinates: itime, psi_dim
Data variables:
    psi      (itime, psi_dim) float32 0.0 1.0 2.0 3.0 ... 2e+05 2e+05 2e+05

In [22]:
# Accesing tensorised data

ds_disk = xr.open_dataset("tensorisation.nc") 
ds_disk.psi[0].as_numpy() # Padding values can be seen there, I haven't hound how to remove them.

<xarray.DataArray 'psi' (psi_dim: 200000)>
array([0.00000e+00, 1.00000e+00, 2.00000e+00, ..., 9.96921e+36,
       9.96921e+36, 9.96921e+36], dtype=float32)
Dimensions without coordinates: psi_dim

In [23]:
%%timeit
# Reading test on the tensorisation data
ds_disk = xr.open_dataset("tensorisation.nc")
assert((ds_disk.psi[1].as_numpy() == GRID_2).all())

1e+03 ms ± 136 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
# Reading test on the variabilisation data
ds_disk = xr.open_dataset("variabilisation.nc") 
assert((ds_disk.time_slice_1__profiles_1d__psi.as_numpy() == GRID_1).all())
assert((ds_disk.time_slice_2__profiles_1d__psi.as_numpy() == GRID_2).all())

3.82 ms ± 787 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%%timeit
# Reading test
ds_disk = xr.open_dataset("compresion.nc") 

assert((ds_disk.psi_value[: int(ds_disk.psi_index[0])]== GRID_1).all())
assert((ds_disk.psi_value[int(ds_disk.psi_index[0]) : int(ds_disk.psi_index[1])] == GRID_2).all())

3.58 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Modifying and saving an NetCDF file with Xarray

In [26]:
ds_disk = xr.open_dataset("variabilisation.nc") 
ds_disk.time_slice_1__profiles_1d__psi[:] = np.array(GRID_1[::-1]) 
ds_disk.time_slice_2__profiles_1d__psi[:] = np.array(GRID_2[::-1])

# Add a new variable
ds_disk["time_slice_3__profiles_1d__psi"] =  np.arange(0,10)

# Modify an existing variable
ds_disk["time_slice_2__profiles_1d__psi"] =  xr.DataArray(np.arange(0,100_000), dims=["size1"])

ds_disk.to_netcdf("variabilisation2.nc", group="time_slice")